# Building a regression model for predicting house sale prices

In [ ]:
import pickle
import pathlib

import numpy as np
import pandas as pd

In [ ]:
DATA_DIR = pathlib.Path.cwd() / 'data'
print(DATA_DIR)

/Users/enriccogemha/Developer/HousePriceRegressor/data


In [ ]:
clean_data_path = DATA_DIR / 'processed' / 'ames_model_data.pkl'

In [ ]:
with open(clean_data_path, 'rb') as file:
    model_data = pickle.load(file)

## Train-test splitting

In [ ]:
X = model_data.drop(columns=['SalePrice']).copy()
y = model_data['SalePrice'].copy()

In [ ]:
X, y

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
RANDOM_SEED = 42  # Any number here, really.

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=RANDOM_SEED,
)


In [ ]:
X.shape, Xtrain.shape, Xtest.shape

In [ ]:
y.shape, ytrain.shape, ytest.shape

## First experiment: scaling and oob

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

float64_columns = Xtrain.select_dtypes('float64').columns

col = ColumnTransformer(
    [
        ('scale', StandardScaler(), float64_columns),
    ],
    remainder='passthrough',
)

col.fit(Xtrain)

Xtrain_scaled = col.transform(Xtrain)
Xtest_scaled = col.transform(Xtest)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

param_grid = {
    'n_estimators': [800],
    'max_depth': [15],
    'min_samples_split': [2],
    'min_samples_leaf': [1],
}

grid_search = GridSearchCV(
    RandomForestRegressor(random_state=RANDOM_SEED, oob_score=True),
    param_grid=param_grid,
    cv=5,
    n_jobs=-1,
    verbose=2,
)

grid_search.fit(Xtrain_scaled, ytrain)


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END max_depth=15, min_samples_leaf=1, min_samples_split=2, n_estimators=800; total time= 1.3min
[CV] END max_depth=15, min_samples_leaf=1, min_samples_split=2, n_estimators=800; total time= 1.4min
[CV] END max_depth=15, min_samples_leaf=1, min_samples_split=2, n_estimators=800; total time= 1.4min
[CV] END max_depth=15, min_samples_leaf=1, min_samples_split=2, n_estimators=800; total time= 1.4min
[CV] END max_depth=15, min_samples_leaf=1, min_samples_split=2, n_estimators=800; total time= 1.4min


GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [15], 'min_samples_leaf': [1],
                         'min_samples_split': [2], 'n_estimators': [800]},
             verbose=2)

In [ ]:
print(f'Best parameters: {grid_search.best_params_}')

In [30]:
#get cross validation rmse of random forest
rmse = np.sqrt(mean_squared_error(ytrain, grid_search.predict(Xtrain_scaled)))
error_percent = 100 * (10**rmse - 1)
print(f'Average error is {error_percent:.2f}%')

In [ ]:
val = grid_search.best_estimator_.predict(Xtest_scaled)
rmse = np.sqrt(mean_squared_error(ytest, val))
error_percent = 100 * (10**rmse - 1)
print(f'Average error is {error_percent:.2f}%')

## Second experiment: skewness

In [ ]:
from scipy.stats import skew

skewness = Xtrain.select_dtypes(np.number).apply(skew)

skewness

In [ ]:
skewness = skewness[abs(skewness) > 3]

skew_features = Xtrain[skewness.index]

skew_features.columns

In [ ]:
def log_tf(feature):
    return np.log1p(feature)

Xtrain_skew = Xtrain.copy()
Xtest_skew = Xtest.copy()

Xtrain_skew[skew_features.columns] = Xtrain_skew[skew_features.columns].apply(log_tf)
Xtest_skew[skew_features.columns] = Xtest_skew[skew_features.columns].apply(log_tf)

In [ ]:
col.fit(Xtrain_skew)

Xtrain_scaled = col.transform(Xtrain_skew)
Xtest_scaled = col.transform(Xtest_skew)

In [ ]:
grid_search.fit(Xtrain_scaled, ytrain)

In [ ]:
# it isnt possible to get rmse score of grid search because RF use r^2 as score metric
rmse = np.sqrt(mean_squared_error(ytrain, grid_search.predict(Xtrain_scaled)))
error_percent = 100 * (10**rmse - 1)
print(f'Average error is {error_percent:.2f}%')

In [ ]:
val = grid_search.best_estimator_.predict(Xtest_scaled)
rmse = np.sqrt(mean_squared_error(ytest, val))
error_percent = 100 * (10**rmse - 1)
print(f'Average error is {error_percent:.2f}%')

## Ainda pior que o baseline